In [ ]:
import mne
from mne.datasets import testing
data_path = testing.data_path()
raw_fname = data_path + '/MEG/sample/sample_audvis_trunc_raw.fif'
subjects_dir = data_path + '/subjects'
subject = 'sample'
trans = data_path + '/MEG/sample/sample_audvis_trunc-trans.fif'
info = mne.io.read_info(raw_fname)
mne.viz.set_3d_backend('notebook')
fig = mne.viz.plot_alignment(info, trans, subject=subject, dig=True,
                             meg=['helmet', 'sensors'], subjects_dir=subjects_dir,
                             surfaces=['head-dense'])
assert fig.display is not None

In [ ]:
from contextlib import contextmanager
import os
from numpy.testing import assert_allclose
from ipywidgets import Button
import matplotlib.pyplot as plt
import mne
from mne.datasets import testing
data_path = testing.data_path()
sample_dir = os.path.join(data_path, 'MEG', 'sample')
subjects_dir = os.path.join(data_path, 'subjects')
fname_stc = os.path.join(sample_dir, 'sample_audvis_trunc-meg')
stc = mne.read_source_estimate(fname_stc, subject='sample')
initial_time = 0.13
mne.viz.set_3d_backend('notebook')
brain_class = mne.viz.get_brain_class()


@contextmanager
def interactive(on):
    old = plt.isinteractive()
    plt.interactive(on)
    try:
        yield
    finally:
        plt.interactive(old)

with interactive(False):
    brain = stc.plot(subjects_dir=subjects_dir, initial_time=initial_time,
                     clim=dict(kind='value', pos_lims=[3, 6, 9]),
                     time_viewer=True,
                     show_traces=True,
                     hemi='lh', size=300)
    assert isinstance(brain, brain_class)
    assert brain.notebook
    assert brain._renderer.figure.display is not None
    brain._update()
    total_number_of_buttons = len([k for k in brain.actions.keys() if '_field' not in k])
    number_of_buttons = 0
    for action in brain.actions.values():
        if isinstance(action, Button):
            action.click()
            number_of_buttons += 1
    assert number_of_buttons == total_number_of_buttons
    img_nv = brain.screenshot()
    assert img_nv.shape == (300, 300, 3), img_nv.shape
    img_v = brain.screenshot(time_viewer=True)
    assert img_v.shape[1:] == (300, 3), img_v.shape
    # XXX This rtol is not very good, ideally would be zero
    assert_allclose(img_v.shape[0], img_nv.shape[0] * 1.25, err_msg=img_nv.shape, rtol=0.1)
    brain.close()

In [ ]:
import mne
mne.viz.set_3d_backend('notebook')
rend = mne.viz.create_3d_figure(size=(100, 100), scene=False)
fig = rend.scene()
mne.viz.set_3d_title(fig, 'Notebook testing')
mne.viz.set_3d_view(fig, 200, 70, focalpoint=[0, 0, 0])
assert fig.display is None
rend.show()
total_number_of_buttons = len([k for k in rend.actions.keys() if '_field' not in k])
number_of_buttons = 0
for action in rend.actions.values():
    if isinstance(action, Button):
        action.click()
        number_of_buttons += 1
assert number_of_buttons == total_number_of_buttons
assert fig.display is not None